In [22]:
import torch, os, copy, time
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset
from utils import *
import torch.nn.functional as F

In [23]:
class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label


class LocalUpdate(object):
    def __init__(self, dataset, idxs):
        self.trainloader, self.validloader, self.testloader = self.train_val_test(dataset, list(idxs))
        # Default criterion set to NLL loss function
        self.criterion = nn.NLLLoss().to(DEVICE)
        # self.criterion = F.cross_entropy
         

    def train_val_test(self, dataset, idxs):
        """
        Returns train, validation and test dataloaders for a given dataset
        and user indexes.
        """
        # split indexes for train, validation, and test (80, 10, 10)
        idxs_train = idxs[:int(0.8*len(idxs))]
        idxs_val = idxs[int(0.8*len(idxs)):int(0.9*len(idxs))]
        idxs_test = idxs[int(0.9*len(idxs)):]

        trainloader = DataLoader(DatasetSplit(dataset, idxs_train),
                                 batch_size=BATCH_SIZE, shuffle=True)
        validloader = DataLoader(DatasetSplit(dataset, idxs_val),
                                 batch_size=int(len(idxs_val)/10), shuffle=False)
        testloader = DataLoader(DatasetSplit(dataset, idxs_test),
                                batch_size=int(len(idxs_test)/10), shuffle=False)
        return trainloader, validloader, testloader

    def update_weights(self, model, global_round):
        # Set mode to train model
        model.train()
        epoch_loss = []

        # Set optimizer for the local updates
        if OPTIMIZER == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE,
                                        momentum=0.5)
        elif OPTIMIZER == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE,
                                         weight_decay=1e-4)

        for iter in range(LOCAL_EPOCHS):
            batch_loss = []
            for batch_idx, (images, labels) in enumerate(self.trainloader):
                images, labels = images.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
                # we need to set the gradients to zero before starting to do backpropragation 
                # because PyTorch accumulates the gradients on subsequent backward passes. 
                # This is convenient while training RNNs
                model.zero_grad()
                
                log_probs = model(images)
                loss = self.criterion(log_probs, labels)
                loss.backward()
                optimizer.step()

                if batch_idx % 10 == 0:
                    print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        global_round, iter, batch_idx * len(images),
                        len(self.trainloader.dataset),
                        100. * batch_idx / len(self.trainloader), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))

        return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

    def inference(self, model):
        """ Returns the inference accuracy and loss.
        """

        model.eval()
        loss, total, correct = 0.0, 0.0, 0.0

        for batch_idx, (images, labels) in enumerate(self.testloader):
            images, labels = images.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)

            # Inference
            outputs = model(images)
            batch_loss = self.criterion(outputs, labels)
            loss += batch_loss.item()

            # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            correct += torch.sum(torch.eq(pred_labels, labels)).item()
            total += len(labels)

        accuracy = correct/total
        return accuracy, loss


def test_inference(model, test_dataset):
    """ Returns the test accuracy and loss.
    """

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0

    criterion = nn.NLLLoss().to(DEVICE)
    testloader = DataLoader(test_dataset, batch_size=128,
                            shuffle=False)

    for batch_idx, (images, labels) in enumerate(testloader):
        images = images.to(DEVICE)
        labels =  labels.to(DEVICE).type(torch.LongTensor)
        # Inference
        outputs = model(images)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        correct += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)

    accuracy = correct/total
    return accuracy, loss

In [24]:
class logReg(torch.nn.Module):
    # logistic regression 
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.linear = torch.nn.Linear(num_features, num_classes)

    def forward(self, x):
        logits = self.linear(x.float())
        probas = torch.sigmoid(logits)
        return probas.type(torch.FloatTensor)

global_model = logReg(num_features=NUM_FEATURES, num_classes=2)
global_weights = global_model.state_dict()

In [25]:
def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg

In [ ]:
# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0

for epoch in tqdm(range(NUM_EPOCHS)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(FRAC * NUM_CLIENTS), 1) # the number of clients to be chosen in each epoch
    idxs_users = np.random.choice(range(NUM_CLIENTS), m, replace=False)

    for idx in idxs_users:
        local_model = LocalUpdate(dataset=train_dataset,
                                    idxs=clients_idx[idx])
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # update global weights
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    global_model.eval()
    for c in range(NUM_CLIENTS):
        local_model = LocalUpdate(dataset=train_dataset,
                                    idxs=clients_idx[idx])
        acc, loss = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

# Test inference after completion of training
test_acc, test_loss = test_inference(global_model, test_dataset)

print(f' \n Results after {EPOCHS} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Saving the objects train_loss and train_accuracy:
# file_name = '../save/objects/{}_{}_{}_C[{}]_iid[{}]_E[{}]_B[{}].pkl'.\
#     format(args.dataset, args.model, args.epochs, args.frac, args.iid,
#             args.local_ep, args.local_bs)

# with open(file_name, 'wb') as f:
#     pickle.dump([train_loss, train_accuracy], f)

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))






  0%|          | 0/20 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/16103 (0%)]	Loss: -0.506022
| Global Round : 0 | Local Epoch : 0 | [1280/16103 (8%)]	Loss: -0.697538
| Global Round : 0 | Local Epoch : 0 | [2560/16103 (16%)]	Loss: -0.808972
| Global Round : 0 | Local Epoch : 0 | [3840/16103 (24%)]	Loss: -0.866907
| Global Round : 0 | Local Epoch : 0 | [5120/16103 (32%)]	Loss: -0.888800
| Global Round : 0 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: -0.912493
| Global Round : 0 | Local Epoch : 0 | [7680/16103 (48%)]	Loss: -0.929048
| Global Round : 0 | Local Epoch : 0 | [8960/16103 (56%)]	Loss: -0.935023
| Global Round : 0 | Local Epoch : 0 | [10240/16103 (63%)]	Loss: -0.949350
| Global Round : 0 | Local Epoch : 0 | [11520/16103 (71%)]	Loss: -0.947600
| Global Round : 0 | Local Epoch : 0 | [12800/16103 (79%)]	Loss: -0.960453
| Global Round : 0 | Local Epoch : 0 | [14080/16103 (87%)]	Loss: -0.963292
| Global Round : 0 | Local Epoch : 0 | [15360/16103 (95%)]	Loss: -0.961813
| Glo

| Global Round : 0 | Local Epoch : 9 | [1280/16103 (8%)]	Loss: -0.996789
| Global Round : 0 | Local Epoch : 9 | [2560/16103 (16%)]	Loss: -0.996499
| Global Round : 0 | Local Epoch : 9 | [3840/16103 (24%)]	Loss: -0.996958
| Global Round : 0 | Local Epoch : 9 | [5120/16103 (32%)]	Loss: -0.996973
| Global Round : 0 | Local Epoch : 9 | [6400/16103 (40%)]	Loss: -0.996106
| Global Round : 0 | Local Epoch : 9 | [7680/16103 (48%)]	Loss: -0.996302
| Global Round : 0 | Local Epoch : 9 | [8960/16103 (56%)]	Loss: -0.997105
| Global Round : 0 | Local Epoch : 9 | [10240/16103 (63%)]	Loss: -0.997032
| Global Round : 0 | Local Epoch : 9 | [11520/16103 (71%)]	Loss: -0.997044
| Global Round : 0 | Local Epoch : 9 | [12800/16103 (79%)]	Loss: -0.996772
| Global Round : 0 | Local Epoch : 9 | [14080/16103 (87%)]	Loss: -0.997310
| Global Round : 0 | Local Epoch : 9 | [15360/16103 (95%)]	Loss: -0.996892
| Global Round : 0 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.501264
| Global Round : 0 | Local Epoch : 0 | 





  5%|▌         | 1/20 [00:06<01:59,  6.27s/it]

| Global Round : 0 | Local Epoch : 9 | [7680/9945 (77%)]	Loss: -0.993618
| Global Round : 0 | Local Epoch : 9 | [8960/9945 (90%)]	Loss: -0.994389

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.994272
| Global Round : 1 | Local Epoch : 0 | [1280/9945 (13%)]	Loss: -0.993891
| Global Round : 1 | Local Epoch : 0 | [2560/9945 (26%)]	Loss: -0.993239
| Global Round : 1 | Local Epoch : 0 | [3840/9945 (38%)]	Loss: -0.993586
| Global Round : 1 | Local Epoch : 0 | [5120/9945 (51%)]	Loss: -0.993876
| Global Round : 1 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: -0.994895
| Global Round : 1 | Local Epoch : 0 | [7680/9945 (77%)]	Loss: -0.993297
| Global Round : 1 | Local Epoch : 0 | [8960/9945 (90%)]	Loss: -0.994781
| Global Round : 1 | Local Epoch : 1 | [0/9945 (0%)]	Loss: -0.994514
| Global Round : 1 | Local Epoch : 1 | [1280/9945 (13%)]	Loss: -0.994921
| Global Round : 1 | Local Epoch : 1 | [2560/9945 (26%)]	Loss: -0.993921
| Global Round : 1 | Local

...

| Global Round : 2 | Local Epoch : 9 | [8960/9945 (90%)]	Loss: -0.998294

 | Global Training Round : 4 |

| Global Round : 3 | Local Epoch : 0 | [0/16103 (0%)]	Loss: -0.998923
| Global Round : 3 | Local Epoch : 0 | [1280/16103 (8%)]	Loss: -0.998931
| Global Round : 3 | Local Epoch : 0 | [2560/16103 (16%)]	Loss: -0.998807
| Global Round : 3 | Local Epoch : 0 | [3840/16103 (24%)]	Loss: -0.998786
| Global Round : 3 | Local Epoch : 0 | [5120/16103 (32%)]	Loss: -0.998817
| Global Round : 3 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: -0.998810
| Global Round : 3 | Local Epoch : 0 | [7680/16103 (48%)]	Loss: -0.998806
| Global Round : 3 | Local Epoch : 0 | [8960/16103 (56%)]	Loss: -0.998884
| Global Round : 3 | Local Epoch : 0 | [10240/16103 (63%)]	Loss: -0.998707
| Global Round : 3 | Local Epoch : 0 | [11520/16103 (71%)]	Loss: -0.998907
| Global Round : 3 | Local Epoch : 0 | [12800/16103 (79%)]	Loss: -0.998887
| Global Round : 3 | Local Epoch : 0 | [14080/16103 (87%)]	Loss: -0.998700
| Globa

| Global Round : 3 | Local Epoch : 8 | [12800/16103 (79%)]	Loss: -0.999320
| Global Round : 3 | Local Epoch : 8 | [14080/16103 (87%)]	Loss: -0.998946
| Global Round : 3 | Local Epoch : 8 | [15360/16103 (95%)]	Loss: -0.999108
| Global Round : 3 | Local Epoch : 9 | [0/16103 (0%)]	Loss: -0.999148
| Global Round : 3 | Local Epoch : 9 | [1280/16103 (8%)]	Loss: -0.999231
| Global Round : 3 | Local Epoch : 9 | [2560/16103 (16%)]	Loss: -0.999192
| Global Round : 3 | Local Epoch : 9 | [3840/16103 (24%)]	Loss: -0.999140
| Global Round : 3 | Local Epoch : 9 | [5120/16103 (32%)]	Loss: -0.999197
| Global Round : 3 | Local Epoch : 9 | [6400/16103 (40%)]	Loss: -0.999094
| Global Round : 3 | Local Epoch : 9 | [7680/16103 (48%)]	Loss: -0.999075
| Global Round : 3 | Local Epoch : 9 | [8960/16103 (56%)]	Loss: -0.999072
| Global Round : 3 | Local Epoch : 9 | [10240/16103 (63%)]	Loss: -0.999064
| Global Round : 3 | Local Epoch : 9 | [11520/16103 (71%)]	Loss: -0.999138
| Global Round : 3 | Local Epoch : 9 |





 20%|██        | 4/20 [00:25<01:39,  6.21s/it]

| Global Round : 3 | Local Epoch : 9 | [7680/9945 (77%)]	Loss: -0.998324
| Global Round : 3 | Local Epoch : 9 | [8960/9945 (90%)]	Loss: -0.998101
 
Avg Training Stats after 4 global rounds:
Training Loss : -0.9919940031637349
Train Accuracy: 73.07% 


 | Global Training Round : 5 |

| Global Round : 4 | Local Epoch : 0 | [0/16103 (0%)]	Loss: -0.999144
| Global Round : 4 | Local Epoch : 0 | [1280/16103 (8%)]	Loss: -0.999071
| Global Round : 4 | Local Epoch : 0 | [2560/16103 (16%)]	Loss: -0.999313
| Global Round : 4 | Local Epoch : 0 | [3840/16103 (24%)]	Loss: -0.999157
| Global Round : 4 | Local Epoch : 0 | [5120/16103 (32%)]	Loss: -0.999093
| Global Round : 4 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: -0.999135
| Global Round : 4 | Local Epoch : 0 | [7680/16103 (48%)]	Loss: -0.999232
| Global Round : 4 | Local Epoch : 0 | [8960/16103 (56%)]	Loss: -0.998876
| Global Round : 4 | Local Epoch : 0 | [10240/16103 (63%)]	Loss: -0.998971
| Global Round : 4 | Local Epoch : 0 | [11520/16103 (7

| Global Round : 4 | Local Epoch : 8 | [5120/16103 (32%)]	Loss: -0.999046
| Global Round : 4 | Local Epoch : 8 | [6400/16103 (40%)]	Loss: -0.999359
| Global Round : 4 | Local Epoch : 8 | [7680/16103 (48%)]	Loss: -0.999369
| Global Round : 4 | Local Epoch : 8 | [8960/16103 (56%)]	Loss: -0.999370
| Global Round : 4 | Local Epoch : 8 | [10240/16103 (63%)]	Loss: -0.999128
| Global Round : 4 | Local Epoch : 8 | [11520/16103 (71%)]	Loss: -0.999261
| Global Round : 4 | Local Epoch : 8 | [12800/16103 (79%)]	Loss: -0.999368
| Global Round : 4 | Local Epoch : 8 | [14080/16103 (87%)]	Loss: -0.999458
| Global Round : 4 | Local Epoch : 8 | [15360/16103 (95%)]	Loss: -0.999237
| Global Round : 4 | Local Epoch : 9 | [0/16103 (0%)]	Loss: -0.999391
| Global Round : 4 | Local Epoch : 9 | [1280/16103 (8%)]	Loss: -0.999216
| Global Round : 4 | Local Epoch : 9 | [2560/16103 (16%)]	Loss: -0.999331
| Global Round : 4 | Local Epoch : 9 | [3840/16103 (24%)]	Loss: -0.999379
| Global Round : 4 | Local Epoch : 9 |